# Fantasy Football - Data Wrangling

In this notebook, we integrate fixture, team, Elo, and form data to construct a dataset suitable for training our goal prediction model. Our training dataset encompasses data from the Premier League seasons 2017-2018 and 2018-2019.

In [1]:
import pandas as pd
import warnings
from functools import reduce
import itertools
import numpy as np

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
warnings.filterwarnings("ignore")

## 1) Fixtures and Results

Importing fixture data for 2017-18 and 2018-19 seasons.

In [3]:
raw_fixture_data = pd.read_csv('data/fixture_data.csv').query('season == "2017-2018" | season == "2018-2019"')

In [4]:
raw_fixture_data = (raw_fixture_data
                    .assign(date=pd.to_datetime(raw_fixture_data.date, format='%d/%m/%Y'),
                            time=pd.to_datetime(raw_fixture_data.kickoff_time).dt.strftime('%H:%M'))
                    .sort_values(by='date')
                    .assign(game_id=raw_fixture_data.index+1))

In [5]:
raw_fixture_data.head()

,game_week,week_day,date,kickoff_time,home_team,away_team,home_xg,away_xg,score,season,game_id,time
0,1,Fri,2017-08-11,19:45,Arsenal,Leicester City,2.5,1.5,4–3,2017-2018,1,19:45
1,1,Sat,2017-08-12,12:30,Watford,Liverpool,2.1,2.6,3–3,2017-2018,2,12:30
2,1,Sat,2017-08-12,15:00,Crystal Palace,Huddersfield,1.1,1.5,0–3,2017-2018,3,15:00
3,1,Sat,2017-08-12,15:00,West Brom,Bournemouth,1.3,0.5,1–0,2017-2018,4,15:00
4,1,Sat,2017-08-12,15:00,Chelsea,Burnley,1.5,0.6,2–3,2017-2018,5,15:00


Dividing the score column into home and away goal columns.

In [6]:
raw_fixture_data[['home_goals', 'away_goals']] = raw_fixture_data.score.str.split('–', expand=True)

In [7]:
raw_fixture_data.drop('score', axis=1, inplace=True)

In [8]:
raw_fixture_data.head()

,game_week,week_day,date,kickoff_time,home_team,away_team,home_xg,away_xg,season,game_id,time,home_goals,away_goals
0,1,Fri,2017-08-11,19:45,Arsenal,Leicester City,2.5,1.5,2017-2018,1,19:45,4,3
1,1,Sat,2017-08-12,12:30,Watford,Liverpool,2.1,2.6,2017-2018,2,12:30,3,3
2,1,Sat,2017-08-12,15:00,Crystal Palace,Huddersfield,1.1,1.5,2017-2018,3,15:00,0,3
3,1,Sat,2017-08-12,15:00,West Brom,Bournemouth,1.3,0.5,2017-2018,4,15:00,1,0
4,1,Sat,2017-08-12,15:00,Chelsea,Burnley,1.5,0.6,2017-2018,5,15:00,2,3


To ensure the correct sequencing of game weeks based on the actual dates of matches played, we integrate FPL game weeks that correspond to the specific dates of the matches. This allows us to align the game weeks with the actual scheduling of matches, rather than relying on the pre-season schedule (which will change as games are rescheduled due to tournament clashes ect throughout the season), which we currently have in 'game_week' column.

Importing fpl game weeks.

In [9]:
fpl_game_weeks = pd.read_csv(r'data/fpl_game_week_data.csv')

In [10]:
fpl_game_weeks = (fpl_game_weeks
                  .assign(date=pd.to_datetime(fpl_game_weeks.date, format='%d/%m/%Y'),
                          time=pd.to_datetime(fpl_game_weeks.time).dt.strftime('%H:%M')))

In [11]:
fpl_game_weeks.head()

,season,date,time,fpl_game_week
0,2018-2019,2018-08-10,20:00,1
1,2018-2019,2018-08-11,12:30,1
2,2018-2019,2018-08-11,15:00,1
3,2018-2019,2018-08-11,17:30,1
4,2018-2019,2018-08-12,13:30,1


Add fpl data to fixtures.

In [12]:
match_fixtures = (pd.merge(raw_fixture_data, fpl_game_weeks, on=['date', 'time', 'season'])
                 .sort_values(by='date')
                 .reset_index(drop=True)
                 .drop('kickoff_time', axis=1)
                 .drop_duplicates(subset=['home_team', 'date']))

In [13]:
match_fixtures.head()

,game_week,week_day,date,home_team,away_team,home_xg,away_xg,season,game_id,time,home_goals,away_goals,fpl_game_week
0,1,Fri,2017-08-11,Arsenal,Leicester City,2.5,1.5,2017-2018,1,19:45,4,3,1
1,1,Sat,2017-08-12,Watford,Liverpool,2.1,2.6,2017-2018,2,12:30,3,3,1
2,1,Sat,2017-08-12,Crystal Palace,Huddersfield,1.1,1.5,2017-2018,3,15:00,0,3,1
3,1,Sat,2017-08-12,West Brom,Bournemouth,1.3,0.5,2017-2018,4,15:00,1,0,1
4,1,Sat,2017-08-12,Chelsea,Burnley,1.5,0.6,2017-2018,5,15:00,2,3,1


Checking for incomplete data. Each season has 380 games and we have 2 seasons worth of data meaning we should have 760 (380 * 2) complete rows.

In [14]:
match_fixtures.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 760 entries, 0 to 759
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   game_week      760 non-null    int64         
 1   week_day       760 non-null    object        
 2   date           760 non-null    datetime64[ns]
 3   home_team      760 non-null    object        
 4   away_team      760 non-null    object        
 5   home_xg        760 non-null    float64       
 6   away_xg        760 non-null    float64       
 7   season         760 non-null    object        
 8   game_id        760 non-null    int64         
 9   time           760 non-null    object        
 10  home_goals     760 non-null    object        
 11  away_goals     760 non-null    object        
 12  fpl_game_week  760 non-null    int64         
dtypes: datetime64[ns](1), float64(2), int64(3), object(7)
memory usage: 77.3+ KB


Getting individual rows for each team participating in a match, rather than having a single row per match.

In [15]:
team_fixtures = match_fixtures.copy()

In [16]:
home_teams = team_fixtures.rename(columns={'home_team': 'team', 'away_team': 'opponent', 'home_xg': 'team_xg',
                                           'away_xg': 'opponent_xg', 'home_goals': 'team_goals',
                                           'away_goals': 'opponent_goals'})

In [17]:
away_teams = team_fixtures.rename(columns={'away_team': 'team', 'home_team': 'opponent', 'away_xg': 'team_xg',
                                           'home_xg': 'opponent_xg', 'away_goals': 'team_goals',
                                           'home_goals': 'opponent_goals'})

In [18]:
team_fixtures = pd.concat([home_teams, away_teams], axis=0, ignore_index=True)

In [19]:
team_fixtures.sort_values(by='game_id').reset_index(drop=True).head()

,game_week,week_day,date,team,opponent,team_xg,opponent_xg,season,game_id,time,team_goals,opponent_goals,fpl_game_week
0,1,Fri,2017-08-11,Arsenal,Leicester City,2.5,1.5,2017-2018,1,19:45,4,3,1
1,1,Fri,2017-08-11,Leicester City,Arsenal,1.5,2.5,2017-2018,1,19:45,3,4,1
2,1,Sat,2017-08-12,Liverpool,Watford,2.6,2.1,2017-2018,2,12:30,3,3,1
3,1,Sat,2017-08-12,Watford,Liverpool,2.1,2.6,2017-2018,2,12:30,3,3,1
4,1,Sat,2017-08-12,Crystal Palace,Huddersfield,1.1,1.5,2017-2018,3,15:00,0,3,1


In [20]:
team_fixtures.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1520 entries, 0 to 1519
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   game_week       1520 non-null   int64         
 1   week_day        1520 non-null   object        
 2   date            1520 non-null   datetime64[ns]
 3   team            1520 non-null   object        
 4   opponent        1520 non-null   object        
 5   team_xg         1520 non-null   float64       
 6   opponent_xg     1520 non-null   float64       
 7   season          1520 non-null   object        
 8   game_id         1520 non-null   int64         
 9   time            1520 non-null   object        
 10  team_goals      1520 non-null   object        
 11  opponent_goals  1520 non-null   object        
 12  fpl_game_week   1520 non-null   int64         
dtypes: datetime64[ns](1), float64(2), int64(3), object(7)
memory usage: 154.5+ KB


We now have two data frames, one with match data and one with team data.

## 2) Elo Ratings
The method we will use to measure relative strength levels is the 
Elo rating system, initially devised by physics professor 
Arpad Elo for improving chess player ratings. This 
system can be applied to football 
teams, assigning each team a single rating indicating relative strengths. The 
difference in ratings between two teams serves as a 
predictor of the match outcome.

Adding elo data to fixture data.

In [21]:
team_elo_data = pd.read_csv('data/team_elo_data.csv')

In [22]:
team_elo_data = (team_elo_data
                 .assign(From=pd.to_datetime(team_elo_data.From, format='%d/%m/%Y'),
                         To=pd.to_datetime(team_elo_data.To, format='%d/%m/%Y')))

In [23]:
team_elo_data.head()

,Rank,Club,Country,Level,Elo,From,To
0,8.0,Arsenal,ENG,1,1873.913208,2017-01-02,2017-01-03
1,8.0,Arsenal,ENG,1,1869.305542,2017-01-04,2017-01-04
2,7.0,Arsenal,ENG,1,1869.305542,2017-01-05,2017-01-14
3,7.0,Arsenal,ENG,1,1875.965332,2017-01-15,2017-01-21
4,8.0,Arsenal,ENG,1,1875.965332,2017-01-22,2017-01-22


In [24]:
match_fixtures = (match_fixtures
                  .merge(team_elo_data[['Club', 'To', 'Elo']], right_on=['Club', 'To'], left_on=['home_team', 'date'])
                  .merge(team_elo_data[['Club', 'To', 'Elo']], right_on=['Club', 'To'], left_on=['away_team', 'date'])
                  .rename(columns={'Elo_x': 'home_elo', 'Elo_y': 'away_elo'})
                  .drop(columns=['Club_x', 'To_x', 'Club_y', 'To_y']))

In [25]:
match_fixtures[['date', 'home_team', 'away_team', 'home_elo', 'away_elo']].head()

,date,home_team,away_team,home_elo,away_elo
0,2017-08-11,Arsenal,Leicester City,1848.286499,1716.994873
1,2017-08-12,Watford,Liverpool,1603.668091,1837.415527
2,2017-08-12,Crystal Palace,Huddersfield,1642.862427,1475.799316
3,2017-08-12,West Brom,Bournemouth,1645.131348,1653.757568
4,2017-08-12,Chelsea,Burnley,1909.399658,1628.988403


In [26]:
team_fixtures = (team_fixtures
                 .merge(team_elo_data[['Club', 'To', 'Elo']], right_on=['Club', 'To'], left_on=['team', 'date'])
                 .merge(team_elo_data[['Club', 'To', 'Elo']], right_on=['Club', 'To'], left_on=['opponent', 'date'])
                 .rename(columns={'Elo_x': 'team_elo', 'Elo_y': 'opponent_elo'})
                 .drop(columns=['Club_x', 'To_x', 'Club_y', 'To_y'])
                 .sort_values(by=['game_id'])
                 .astype({'team_goals': 'float', 'opponent_goals': 'float'}))

In [27]:
team_fixtures[['date', 'team', 'opponent', 'team_elo', 'opponent_elo']].head()

,date,team,opponent,team_elo,opponent_elo
0,2017-08-11,Arsenal,Leicester City,1848.286499,1716.994873
760,2017-08-11,Leicester City,Arsenal,1716.994873,1848.286499
761,2017-08-12,Liverpool,Watford,1837.415527,1603.668091
1,2017-08-12,Watford,Liverpool,1603.668091,1837.415527
2,2017-08-12,Crystal Palace,Huddersfield,1642.862427,1475.799316


## 3) Team Form

To generate team form for different game windows (3, 5, and 10 games), we consider both actual goals scored and conceded, as well as expected goals scored and conceded. We will assess the accuracy of using various combinations of game windows and actual/expected data when developing our models.

In [28]:
def get_form(team_fixtures, col_name, group, metric):
    return team_fixtures.assign(**{col_name: lambda x: x.groupby([group])[metric]
                                                        .transform(lambda x: x.rolling(g+1, min_periods=g+1)
                                                                   .sum()
                                                                   .sub(team_fixtures[metric])
                                                                   .div(g))})

In [29]:
games = [3, 5, 10]

In [30]:
for g in games:
    team_fixtures = (
        get_form(team_fixtures, f'team_goals_scored_xg_{g}game_form', 'team', 'team_xg')
        .pipe(get_form, f'team_goals_conceded_xg_{g}game_form', 'team', 'opponent_xg')
        .pipe(get_form, f'opponent_goals_scored_xg_{g}game_form', 'opponent', 'opponent_xg')
        .pipe(get_form, f'opponent_goals_conceded_xg_{g}game_form', 'opponent', 'team_xg')
        .pipe(get_form, f'team_goals_scored_actual_{g}game_form', 'team', 'team_goals')
        .pipe(get_form, f'team_goals_conceded_actual_{g}game_form', 'team', 'opponent_goals')
        .pipe(get_form, f'opponent_goals_scored_actual_{g}game_form', 'opponent', 'opponent_goals')
        .pipe(get_form, f'opponent_goals_conceded_actual_{g}game_form', 'opponent', 'team_goals')
    )

In [31]:
team_fixtures[team_fixtures.team == 'Liverpool'][['date', 'game_week', 'season', 'team', 'opponent', 'team_xg', 'team_goals_scored_xg_3game_form']].head()

,date,game_week,season,team,opponent,team_xg,team_goals_scored_xg_3game_form
761,2017-08-12,1,2017-2018,Liverpool,Watford,2.6,NaN
13,2017-08-19,2,2017-2018,Liverpool,Crystal Palace,2.5,NaN
29,2017-08-27,3,2017-2018,Liverpool,Arsenal,3.1,NaN
791,2017-09-09,4,2017-2018,Liverpool,Manchester City,0.7,2.733333
45,2017-09-16,5,2017-2018,Liverpool,Burnley,2.2,2.100000


Adding elo difference between two teams.

In [32]:
team_fixtures['elodiff'] = team_fixtures.team_elo - team_fixtures.opponent_elo

In [33]:
team_fixtures[['fpl_game_week', 'team', 'opponent', 'team_elo', 'opponent_elo', 'elodiff']].reset_index(drop=True).head()

,fpl_game_week,team,opponent,team_elo,opponent_elo,elodiff
0,1,Arsenal,Leicester City,1848.286499,1716.994873,131.291626
1,1,Leicester City,Arsenal,1716.994873,1848.286499,-131.291626
2,1,Liverpool,Watford,1837.415527,1603.668091,233.747436
3,1,Watford,Liverpool,1603.668091,1837.415527,-233.747436
4,1,Crystal Palace,Huddersfield,1642.862427,1475.799316,167.063111


Adding column showing if team is playing at home or away.

In [34]:
home_away_data = match_fixtures[['game_id', 'home_team']]

In [35]:
team_fixtures = (team_fixtures
                 .merge(home_away_data, on='game_id')
                 .assign(home=lambda x: (x.team == x.home_team).astype(int))
                 .drop('home_team', axis=1))

In [36]:
team_fixtures[['fpl_game_week', 'date', 'team', 'opponent', 'home']].head()

,fpl_game_week,date,team,opponent,home
0,1,2017-08-11,Arsenal,Leicester City,1
1,1,2017-08-11,Leicester City,Arsenal,0
2,1,2017-08-12,Liverpool,Watford,0
3,1,2017-08-12,Watford,Liverpool,1
4,1,2017-08-12,Crystal Palace,Huddersfield,1


Our machine learning model, does not accept NaN values, as a result we need to eliminate all such occurrences. For the chosen form metric, depending on the number of past games considered (3, 5, or 10 game weeks), the initial 3, 5, or 10 game weeks of the 2017-18 season will contain NaN values.

Removing first 5 game weeks of 2017-18 season.

In [37]:
team_fixtures = team_fixtures.query('game_id > 50')

In [38]:
team_fixtures.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1420 entries, 100 to 1519
Data columns (total 41 columns):
 #   Column                                      Non-Null Count  Dtype         
---  ------                                      --------------  -----         
 0   game_week                                   1420 non-null   int64         
 1   week_day                                    1420 non-null   object        
 2   date                                        1420 non-null   datetime64[ns]
 3   team                                        1420 non-null   object        
 4   opponent                                    1420 non-null   object        
 5   team_xg                                     1420 non-null   float64       
 6   opponent_xg                                 1420 non-null   float64       
 7   season                                      1420 non-null   object        
 8   game_id                                     1420 non-null   int64         
 9   time       

Additionally, the first 3, 5, or 10 game weeks for matches involving recently promoted teams will also yield NaN values. These NaN values will be replaced by the 25th and 75th percentiles of the league's goals scored and conceded, as promoted clubs often exhibit comparatively poor performance levels.

Filling promoted team na values with the 25th percentile of goals scored and 75th percentile of goals conceded by teams in league.

In [39]:
team_fixtures[['team_goals_scored_actual_5game_form', 'team_goals_conceded_actual_5game_form']].describe()

,team_goals_scored_actual_5game_form,team_goals_conceded_actual_5game_form
count,1405.000000,1405.000000
mean,1.380214,1.375089
std,0.689342,0.618776
min,0.000000,0.000000
25%,0.800000,1.000000
50%,1.200000,1.400000
75%,1.800000,1.800000
max,4.800000,3.400000


In [40]:
games

[3, 5, 10]

In [41]:
def fill_missing(col, col_2, quant):
    team_fixtures[col].fillna(team_fixtures[col_2].quantile(quant), inplace=True)

In [42]:
for g in games:
    # filling xg form
    fill_missing(f'team_goals_scored_xg_{g}game_form', f'team_goals_scored_xg_{g}game_form', .25)
    fill_missing(f'opponent_goals_scored_xg_{g}game_form', f'team_goals_scored_xg_{g}game_form', .75)
    fill_missing(f'team_goals_conceded_xg_{g}game_form', f'team_goals_scored_xg_{g}game_form', .75)
    fill_missing(f'opponent_goals_conceded_xg_{g}game_form', f'team_goals_scored_xg_{g}game_form', .25)
    # filling actual form
    fill_missing(f'team_goals_scored_actual_{g}game_form', f'team_goals_scored_actual_{g}game_form', .25)
    fill_missing(f'opponent_goals_scored_actual_{g}game_form', f'team_goals_scored_actual_{g}game_form', .75)
    fill_missing(f'team_goals_conceded_actual_{g}game_form', f'team_goals_scored_actual_{g}game_form', .75)
    fill_missing(f'opponent_goals_conceded_actual_{g}game_form', f'team_goals_scored_actual_{g}game_form', .25)

Checking that there are no more NaN values.

In [43]:
team_fixtures.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1420 entries, 100 to 1519
Data columns (total 41 columns):
 #   Column                                      Non-Null Count  Dtype         
---  ------                                      --------------  -----         
 0   game_week                                   1420 non-null   int64         
 1   week_day                                    1420 non-null   object        
 2   date                                        1420 non-null   datetime64[ns]
 3   team                                        1420 non-null   object        
 4   opponent                                    1420 non-null   object        
 5   team_xg                                     1420 non-null   float64       
 6   opponent_xg                                 1420 non-null   float64       
 7   season                                      1420 non-null   object        
 8   game_id                                     1420 non-null   int64         
 9   time       

Amount of games we have data for by team and season. We should see 33 games for teams in 2017-18 seasons as we have removed the first five game weeks. Some teams may only have data for one season if they were promoted or relegated in the 2017-18 season.

In [44]:
team_fixtures.groupby(['team', 'season'])['game_id'].count()

team             season   
Arsenal          2017-2018    33
                 2018-2019    38
Bournemouth      2017-2018    33
                 2018-2019    38
Brighton         2017-2018    33
                 2018-2019    38
Burnley          2017-2018    33
                 2018-2019    38
Cardiff City     2018-2019    38
Chelsea          2017-2018    33
                 2018-2019    38
Crystal Palace   2017-2018    33
                 2018-2019    38
Everton          2017-2018    33
                 2018-2019    38
Fulham           2018-2019    38
Huddersfield     2017-2018    33
                 2018-2019    38
Leicester City   2017-2018    33
                 2018-2019    38
Liverpool        2017-2018    33
                 2018-2019    38
Manchester City  2017-2018    33
                 2018-2019    38
Manchester Utd   2017-2018    33
                 2018-2019    38
Newcastle Utd    2017-2018    33
                 2018-2019    38
Southampton      2017-2018    33
                

Exporting our final df to use for model training.

In [45]:
team_fixtures.to_csv('wrangled_data_final.csv', index=False)